<a href="https://colab.research.google.com/github/mjain2/csci544-group32/blob/main/CNN_LSTM_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Flatten, Dense, Input, Conv1D, LSTM, Concatenate
from tensorflow import keras
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import mean_squared_error
import numpy as np
import gc
import pickle
from keras.layers import Layer
import keras.backend as K

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:

# with open('/content/gdrive/MyDrive/ProjectEmbeddings/test_tok_emb.pkl', 'rb') as f:
#     train_tok_emb = pickle.load(f)

# with open('/content/gdrive/MyDrive/ProjectEmbeddings/train_sent_emb.pkl', 'rb') as f:
#     train_sent_emb = pickle.load(f)

# # with open('/content/gdrive/MyDrive/Colab Notebooks/NLP/Project/feedback-prize-english-language-learning/test_sent_emb.pkl', 'rb') as f:
# #     test_sent_emb = pickle.load(f)

# with open('/content/gdrive/MyDrive/ProjectEmbeddings/test_tok_emb.pkl', 'rb') as f:
#     test_tok_emb = pickle.load(f)

# with open('/content/gdrive/MyDrive/ProjectEmbeddings/y_train.pkl', 'rb') as f:
#     y_train = pickle.load(f)

# with open('/content/gdrive/MyDrive/ProjectEmbeddings/y_test.pkl', 'rb') as f:
#     y_test = pickle.load(f)

# with open('df_train_folds.pkl', 'rb') as f:
#     df_train_folds = pickle.load(f)

In [ ]:

with open('/content/gdrive/MyDrive/Colab Notebooks/NLP/Project/feedback-prize-english-language-learning/train_tok_emb.pkl', 'rb') as f:
    train_tok_emb = pickle.load(f)

with open('/content/gdrive/MyDrive/Colab Notebooks/NLP/Project/feedback-prize-english-language-learning/train_sent_emb.pkl', 'rb') as f:
    train_sent_emb = pickle.load(f)

with open('/content/gdrive/MyDrive/Colab Notebooks/NLP/Project/feedback-prize-english-language-learning/test_sent_emb.pkl', 'rb') as f:
    test_sent_emb = pickle.load(f)

with open('/content/gdrive/MyDrive/Colab Notebooks/NLP/Project/feedback-prize-english-language-learning/test_tok_emb.pkl', 'rb') as f:
    test_tok_emb = pickle.load(f)

with open('/content/gdrive/MyDrive/Colab Notebooks/NLP/Project/feedback-prize-english-language-learning/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('/content/gdrive/MyDrive/Colab Notebooks/NLP/Project/feedback-prize-english-language-learning/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

with open('/content/gdrive/MyDrive/Colab Notebooks/NLP/Project/feedback-prize-english-language-learning/df_train_folds.pkl', 'rb') as f:
    df_train_folds = pickle.load(f)

In [ ]:
def get_optimizer():
  adam = tf.keras.optimizers.Adam(learning_rate=0.001)
  return adam

def mcrmse(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=0)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=0)

In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        print(self.W.shape)
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros") 
        print(self.b.shape)       
        super(attention, self).build(input_shape)

    def call(self,x):
        temp_dot = K.dot(x,self.W)
        temp_sum = temp_dot + self.b
        temp_tanh = K.tanh(temp_sum)
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
def LSTM_CNN1D(feat_len, seq_len, embedding_len):
    drop_lstm = 0.25
    drop_dense = 0.25
    num_lstm=150
    input_1 = Input(shape=(seq_len, embedding_len,),name = 'input_comment_1')
    conv_1_1 = Conv1D(64,3,strides=1, padding='same',activation='relu')(input_1)
    lstm_1_1 = LSTM(64,dropout=drop_lstm,return_sequences=True,dtype=tf.float32)(conv_1_1)
    concate_1 = keras.layers.Concatenate(axis=-1)([conv_1_1, lstm_1_1])
    flatten_1 = Flatten()(lstm_1_1)
    attention_output_1 = Flatten()(attention()(concate_1))

    # creating layers for parent comment data
    input_2 = Input(shape=(seq_len, embedding_len,),name = 'input_comment_2')
    conv_1_2 = Conv1D(128,3,strides=1, padding='same',activation='relu')(input_2)
    lstm_1_2 =LSTM(128,dropout=drop_lstm,return_sequences=True,dtype=tf.float32)(conv_1_2)
    concate_2 = keras.layers.Concatenate(axis=-1)([conv_1_2, lstm_1_2])
    flatten_2 = Flatten()(lstm_1_2)

    attention_output_2 = Flatten()(attention()(concate_2))
    
    # creating further layers
    input_dense = Input(shape=(feat_len,),name = 'input_comment')
    dense_num_layer = Dense(128, activation = "relu")(input_dense)

    #concatenated_layer = keras.layers.concatenate([Flatten()(concate_1),attention_output_1,Flatten()(concate_2),attention_output_2,dense_num_layer],axis=-1)
    # concatenated_layer = keras.layers.concatenate([attention_output_1,flatten_1,attention_output_2,flatten_2,dense_num_layer],axis=-1)

    concatenated_layer = keras.layers.concatenate([attention_output_1,attention_output_2, dense_num_layer],axis=-1)
    
    x = Dense(128, activation = 'relu',kernel_initializer = glorot_uniform(seed=42))(concatenated_layer)
    x = BatchNormalization()(x)
    x = Dense(64, activation = 'relu',kernel_initializer = glorot_uniform(seed=42))(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation = 'relu',kernel_initializer = glorot_uniform(seed=42))(x)
    x = BatchNormalization()(x)
    x = Dense(16, activation = 'relu',kernel_initializer= glorot_uniform(seed=42))(x)
    #x = Dropout(drop_dense)(x)
    output = Dense(6,activation='linear')(x)
    
    model = Model(inputs = [input_dense, input_1, input_2], outputs = [output])
    model.compile(optimizer=get_optimizer(), loss = mcrmse, metrics = mcrmse)
    
    return model

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_mcrmse', factor = 0.25, patience = 2, verbose = 1)
earlystop = EarlyStopping(monitor = 'val_mcrmse',  mode="min",min_delta = 0, patience = 25,verbose = 1)

callbacks = [reduce_lr,earlystop]

In [ ]:
model=LSTM_CNN1D(train_sent_emb.shape[1], train_tok_emb.shape[1], train_tok_emb.shape[2])
model.summary()

(128, 1)
(640, 1)
(256, 1)
(640, 1)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_comment_1 (InputLayer)   [(None, 640, 768)]   0           []                               
                                                                                                  
 input_comment_2 (InputLayer)   [(None, 640, 768)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 640, 64)      147520      ['input_comment_1[0][0]']        
                                                                                                  
 conv1d_1 (Conv1D)              (None, 640, 128)     295040      ['input_comment_2[0][0]']        
                                                          

In [ ]:

target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
preds = []
scores = []
def comp_score(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)

def train(all_train_text_feats, all_train_token_embedding, dftr, FOLDS=2):
  #for fold in tqdm(range(FOLDS),total=FOLDS):
  for fold in range(FOLDS):
      print('#'*25)
      print('### Fold',fold+1)
      print('#'*25)
      
      dftr_ = dftr[dftr["FOLD"]!=fold]
      dfev_ = dftr[dftr["FOLD"]==fold]
      
      tr_text_feats = all_train_text_feats[list(dftr_.index),:]
      tr_token_feats = all_train_token_embedding[list(dftr_.index),:]
      y_train = dftr_.loc[list(dftr_.index), target_cols].values
      
      ev_text_feats = all_train_text_feats[list(dfev_.index),:]
      ev_token_feats = all_train_token_embedding[list(dfev_.index),:]
      y_test = dfev_.loc[list(dfev_.index), target_cols].values
      
      ev_preds = np.zeros((len(ev_text_feats),6))
      # test_preds = np.zeros((len(te_text_feats),6))
      
      hitory=model.fit(x=[tr_text_feats, tr_token_feats, tr_token_feats],
                  y=y_train,
                  epochs=15,
                  batch_size=32,
                  validation_data=([ev_text_feats, ev_token_feats, ev_token_feats], y_test),
                  callbacks=callbacks)
      
      ev_preds = model.predict([ev_text_feats, ev_token_feats, ev_token_feats])
      # test_preds = model.predict([te_text_feats, te_token_embeddings, te_token_embeddings])
      print()
      score = comp_score(dfev_[target_cols].values,ev_preds)
      scores.append(score)
      print("Fold : {} RSME score: {}".format(fold+1,score))
      # preds.append(test_preds)
      del dftr_
      del dfev_ 
      del tr_text_feats 
      del tr_token_feats 
      del y_train
      del ev_text_feats 
      del ev_token_feats 
      del y_test
      
      gc.collect()
      
  print('#'*25)
  print('Overall CV RSME =',np.mean(scores))
  return model

In [ ]:
model = train(train_sent_emb, train_tok_emb, df_train_folds)

#########################
### Fold 1
#########################
Epoch 1/15
79/79 [==============================] - 26s 139ms/step - loss: 2.2950 - mcrmse: 2.2885 - val_loss: 2.2755 - val_mcrmse: 2.2735 - lr: 0.0010
Epoch 2/15
79/79 [==============================] - 8s 99ms/step - loss: 0.7717 - mcrmse: 0.7693 - val_loss: 0.9578 - val_mcrmse: 0.9578 - lr: 0.0010
Epoch 3/15
79/79 [==============================] - 8s 100ms/step - loss: 0.5556 - mcrmse: 0.5553 - val_loss: 0.8490 - val_mcrmse: 0.8467 - lr: 0.0010
Epoch 4/15
79/79 [==============================] - 8s 99ms/step - loss: 0.5378 - mcrmse: 0.5378 - val_loss: 0.6483 - val_mcrmse: 0.6464 - lr: 0.0010
Epoch 5/15
79/79 [==============================] - 8s 99ms/step - loss: 0.5201 - mcrmse: 0.5224 - val_loss: 0.5751 - val_mcrmse: 0.5723 - lr: 0.0010
Epoch 6/15
79/79 [==============================] - 8s 99ms/step - loss: 0.5197 - mcrmse: 0.5240 - val_loss: 0.5210 - val_mcrmse: 0.5191 - lr: 0.0010
Epoch 7/15
79/79 [================

In [ ]:
preds = model.predict([test_sent_emb, test_tok_emb, test_tok_emb])
print(comp_score(np.array(y_test), preds))

25/25 [==============================] - 1s 53ms/step
0.4821823667907146
